# 多視点画像表示

Open3Dを用いて多視点画像を表示するプログラムを紹介します．
ここでは，LightField Descriptor，およびMVCNNやRotationNetで用いられているように，正十二面体の全$20$個の頂点上に仮想カメラを配置して画像を取得します．
まずはじめに，各頂点の3次元座標を変数`vertices`に格納します．

In [1]:
import open3d as o3d
import numpy as np

phi = (1+np.sqrt(5))/2
vertices = np.asarray([
    [1, 1, 1],
    [1, 1, -1],
    [1, -1, 1],
    [1, -1, -1],
    [-1, 1, 1],
    [-1, 1, -1],
    [-1, -1, 1],
    [-1, -1, -1],
    [0, 1/phi, phi],
    [0, 1/phi, -phi],
    [0, -1/phi, phi],
    [0, -1/phi, -phi],
    [phi, 0, 1/phi],
    [phi, 0, -1/phi],
    [-phi, 0, 1/phi],
    [-phi, 0, -1/phi],
    [1/phi, phi, 0],
    [-1/phi, phi, 0],
    [1/phi, -phi, 0],
    [-1/phi, -phi, 0]
])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


次に，各頂点に置かれたカメラからの視点で点群データを表示させるためのコールバック関数を定義します．

In [2]:
i = 0
ROTATION_RADIAN_PER_PIXEL = 0.003
def rotate_view(vis):
    global i
    if i >= vertices.shape[0]:
        vis.close()
        return False
    vis.reset_view_point(True)
    ctr = vis.get_view_control()
    az = np.arctan2(vertices[i,1], vertices[i,0]) 
    el = np.arctan2(vertices[i,2], np.sqrt(vertices[i,0] * vertices[i,0] + vertices[i,1] * vertices[i,1]))
    ctr.rotate(-az/ROTATION_RADIAN_PER_PIXEL, el/ROTATION_RADIAN_PER_PIXEL)
    i += 1
    return False

9行目で$i$番目の頂点の方位角（azimuth），10行目で$i$番目の頂点の高度（elevation）を計算し，
11行目で視点を回転させています．
`ROTATION_RADIAN_PER_PIXEL`は，画像1ピクセルの回転角（ラジアン）を表しており，Open3Dではこの値が$0.003$と定義されています．
11行目の関数の引数はピクセル数で与える必要があるため，先に求めた方位角と高度を`ROTATION_RADIAN_PER_PIXEL`で除算しています．

次に，Bunny.plyのメッシュデータをファイルから読み込みます．

In [3]:
filename = "../3rdparty/Open3D/examples/test_data/Bunny.ply"
pcd = o3d.io.read_point_cloud(filename)

最後に，先に定義したコールバック関数を用いて読み込んだ点群ファイルを描画します．

In [4]:
key_to_callback = {}
key_to_callback[ord(" ")] = rotate_view
o3d.visualization.draw_geometries_with_key_callbacks([pcd], key_to_callback)

スペースキーを一回押すと，次のカメラ視点から見た点群データに表示が切り替わります．
$20$個全てのカメラ視点からのデータを表示すると，プログラムを終了します．